In [1]:
import time
from functools import wraps


def timeit(fn):
    """计算函数运行时间的装饰器"""

    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = time.time()
        result = fn(*args, **kwargs)
        t2 = time.time()
        print(f"@timeit: {fn.__name__} took {t2 - t1: .5f} s")
        return result

    return measure_time

In [2]:
import cv2
import numpy as np


class InsulatingDamageDetect:
    def findMax(self, contours):
        result = contours[0]
        for contour in contours:
            if cv2.contourArea(result) < cv2.contourArea(contour):
                result = contour
        return result

    @timeit
    def process_image(self, image):
        #1.提取白色部分，转二值图
        image = cv2.multiply(image, (1.2, 1.2, 1.2, 1.2))
        minWrite = np.array([150, 150, 150])
        maxWrite = np.array([255, 255, 255])
        mask = cv2.inRange(image, minWrite, maxWrite)
        write = cv2.bitwise_and(image, image, mask=mask)

        write = cv2.cvtColor(write, cv2.COLOR_BGR2GRAY)
        write = cv2.GaussianBlur(write, (5, 5), 0, 0)
        write = cv2.threshold(write, 128, 255, cv2.THRESH_BINARY)[1]


        #2.将白色部分切成小段
        contours = cv2.findContours(write, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
        contour = self.findMax(contours)
        temp = np.zeros(write.shape, np.uint8)
        cv2.drawContours(temp, [contour], -1, 255, -1)

        x, y, w, h = cv2.boundingRect(contour)
        mask = np.zeros(write.shape, np.uint8)
        mask[y:y + h, x + 50:x + w - 50] = 1
        temp = cv2.bitwise_and(temp, mask)
        temp = cv2.bitwise_and(temp,write)

        step = 250
        n = w // step
        step = step + (w % step) // n
        line = []
        for i in range(n):
            mask = np.zeros(write.shape, np.uint8)
            mask[:, x + step * i:x + step * (i + 1)] = 1
            part = cv2.bitwise_and(temp, mask)
            line.append(part)


        #3.找缺陷：找最大轮廓凸集，在最大轮廓上覆盖，用圆形和矩形标记
        draw = np.zeros(write.shape, np.uint8)
        for part in line:
            temp = np.zeros(write.shape, np.uint8)
            contours = cv2.findContours(part, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
            for contour in contours:
                if cv2.contourArea(contour) > 100:
                    cv2.drawContours(temp, [contour], -1, 255, 3)

            contour = self.findMax(contours)
            hull = cv2.convexHull(contour)
            cv2.drawContours(temp, [hull], -1, 0, 15)
            draw = cv2.bitwise_or(draw, temp)

        # plt.imshow(draw,cmap="gray")
        # plt.show()

        suspected_areas_num = 0
        contours = cv2.findContours(draw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
        for contour in contours:
            if cv2.contourArea(contour) > 1000:
                suspected_areas_num = suspected_areas_num + 1
                rect = cv2.minAreaRect(contour)
                points = cv2.boxPoints(rect)
                points = np.int0(points)
                cv2.drawContours(image, [points], 0, (0, 0, 255), 3)
            elif cv2.contourArea(contour) > 50:
                suspected_areas_num = suspected_areas_num + 1
                (x, y), radius = cv2.minEnclosingCircle(contour)
                radius = int(radius)
                x, y = int(x), int(y)
                cv2.circle(image, [x, y], radius, (0, 0, 255), 3)
        return suspected_areas_num <= 0, suspected_areas_num

id = InsulatingDamageDetect()

In [4]:
image = cv2.imread(
            '/Users/zhouxinyu/Code/Python/jupyter/Project/ZF/数据测试/绝缘层污染损伤/数据/损伤/最终测试数据集/00.tif')
flag, num = id.process_image(image)
print("是否污染或损伤: ", not flag, num)

@timeit: process_image took  0.06124 s
是否污染或损伤:  True 5


In [3]:
# 污染2
for i in range(0, 37):
    print(i)
    fileName = '/Users/zhouxinyu/Code/Python/jupyter/Project/ZF/数据测试/绝缘层污染损伤/数据/污染2/'+str(i)+'.jpg'
    image = cv2.imread(fileName)
    flag, num = id.process_image(image)
    print("是否污染: ", not flag, num)

0
@timeit: process_image took  0.06394 s
是否污染:  True 2
1
@timeit: process_image took  0.04869 s
是否污染:  True 2
2
@timeit: process_image took  0.03754 s
是否污染:  True 2
3
@timeit: process_image took  0.03799 s
是否污染:  True 1
4
@timeit: process_image took  0.03563 s
是否污染:  True 3
5
@timeit: process_image took  0.03877 s
是否污染:  True 3
6
@timeit: process_image took  0.03406 s
是否污染:  False 0
7
@timeit: process_image took  0.03807 s
是否污染:  False 0
8
@timeit: process_image took  0.03668 s
是否污染:  False 0
9
@timeit: process_image took  0.03338 s
是否污染:  False 0
10
@timeit: process_image took  0.03422 s
是否污染:  True 2
11
@timeit: process_image took  0.03756 s
是否污染:  True 3
12
@timeit: process_image took  0.03630 s
是否污染:  True 3
13
@timeit: process_image took  0.03404 s
是否污染:  True 3
14
@timeit: process_image took  0.03719 s
是否污染:  True 2
15
@timeit: process_image took  0.03398 s
是否污染:  True 2
16
@timeit: process_image took  0.03395 s
是否污染:  True 2
17
@timeit: process_image took  0.03389 s
是否污染:  True 

In [90]:
# 污染
for i in range(1, 20):
    print(i)
    fileName = '/Users/zhouxinyu/Code/Python/jupyter/Project/ZF/数据测试/绝缘层污染损伤/数据/污染/'+str(i)+'.tif'
    image = cv2.imread(fileName)
    flag, num = id.process_image(image)
    print("是否污染: ", not flag, num)

1
@timeit: process_image took  0.05112 s
是否污染或损伤:  True 6
2
@timeit: process_image took  0.03846 s
是否污染或损伤:  True 9
3
@timeit: process_image took  0.04006 s
是否污染或损伤:  True 7
4
@timeit: process_image took  0.03691 s
是否污染或损伤:  True 5
5
@timeit: process_image took  0.03673 s
是否污染或损伤:  True 1
6
@timeit: process_image took  0.04407 s
是否污染或损伤:  True 7
7
@timeit: process_image took  0.04652 s
是否污染或损伤:  True 9
8
@timeit: process_image took  0.04055 s
是否污染或损伤:  True 2
9
@timeit: process_image took  0.03567 s
是否污染或损伤:  True 1
10
@timeit: process_image took  0.03503 s
是否污染或损伤:  False 0
11
@timeit: process_image took  0.04135 s
是否污染或损伤:  True 7
12
@timeit: process_image took  0.04004 s
是否污染或损伤:  True 10
13
@timeit: process_image took  0.03871 s
是否污染或损伤:  True 12
14
@timeit: process_image took  0.03901 s
是否污染或损伤:  True 9
15
@timeit: process_image took  0.04139 s
是否污染或损伤:  True 2
16
@timeit: process_image took  0.03647 s
是否污染或损伤:  True 4
17
@timeit: process_image took  0.03716 s
是否污染或损伤:  True 3
18


In [9]:
# 损伤
# print(1)
# image = cv2.imread(
#             '/Users/zhouxinyu/Code/Python/jupyter/Project/ZF/数据测试/绝缘层污染损伤/数据/损伤/手动裁剪/1.tif')
# flag, num = id.process_image(image)
# print("是否污染或损伤: ", not flag, num)

for i in range(1, 44):
    print(i)
    fileName = '/Users/zhouxinyu/Code/Python/jupyter/Project/ZF/数据测试/绝缘层污染损伤/数据/损伤/最终测试数据集/'+str(i)+'.tif'
    image = cv2.imread(fileName)
    flag, num = id.process_image(image)
    print("是否损伤: ", not flag, num)

1
@timeit: process_image took  0.03502 s
是否损伤:  True 6
2
@timeit: process_image took  0.03666 s
是否损伤:  True 6
3
@timeit: process_image took  0.03904 s
是否损伤:  True 2
4
@timeit: process_image took  0.03614 s
是否损伤:  True 2
5
@timeit: process_image took  0.03735 s
是否损伤:  True 6
6
@timeit: process_image took  0.03709 s
是否损伤:  True 5
7
@timeit: process_image took  0.03647 s
是否损伤:  True 4
8
@timeit: process_image took  0.03664 s
是否损伤:  True 6
9
@timeit: process_image took  0.03459 s
是否损伤:  True 5
10
@timeit: process_image took  0.03618 s
是否损伤:  True 3
11
@timeit: process_image took  0.03454 s
是否损伤:  True 5
12
@timeit: process_image took  0.03665 s
是否损伤:  True 6
13
@timeit: process_image took  0.03466 s
是否损伤:  True 6
14
@timeit: process_image took  0.03537 s
是否损伤:  True 5
15
@timeit: process_image took  0.03809 s
是否损伤:  True 6
16
@timeit: process_image took  0.03422 s
是否损伤:  True 9
17
@timeit: process_image took  0.03453 s
是否损伤:  True 9
18
@timeit: process_image took  0.03400 s
是否损伤:  True 8
1